In [ ]:
#I am using this notebook as practice to follow along with evolving hockeys tutorial
#on how they calculate their expected goal value. link to the tutorial is here
#https://evolving-hockey.com/blog/a-new-expected-goals-model-for-predicting-goals-in-the-nhl/

In [2]:
import pandas as pd
import numpy as np

In [40]:
#df_import = pd.read_csv('data/washington_sample.csv')
df = pd.read_csv('data/pbp/raw/pbp_query_20172018.csv',low_memory=False)

In [41]:
df.head()

,season,game_id,game_date,session,event_index,game_period,game_seconds,clock_time,event_type,event_description,...,game_strength_state,home_zone,pbp_distance,event_distance,event_angle,home_zonestart,face_index,pen_index,shift_index,pred_goal
0,20172018,2017020001,2017-10-04,R,1,1,0,20:00,PGSTR,,...,EvE,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN
1,20172018,2017020001,2017-10-04,R,2,1,0,20:00,PGEND,,...,EvE,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN
2,20172018,2017020001,2017-10-04,R,3,1,0,20:00,ANTHEM,,...,EvE,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN
3,20172018,2017020001,2017-10-04,R,4,1,0,20:00,PSTR,Period Start- Local time: 6:17 CDT,...,EvE,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN
4,20172018,2017020001,2017-10-04,R,5,1,0,20:00,CHANGE,NaN,...,Ev5,NaN,NaN,NaN,NaN,NaN,0,0,1,NaN


In [42]:
#This page was helpful https://stackoverflow.com/questions/39109045/numpy-where-with-multiple-conditions
#Conditions for extracting the zone
col = 'event_description'
conditions = [df[col].str.contains('Off. Zone', na=False), df[col].str.contains('Def. Zone', na=False), 
              df[col].str.contains('Neu. Zone', na=False)]
choices = ['Off', 'Def','Neu']
fenwick = ['SHOT', 'MISS', 'GOAL']
corsi = ['SHOT', 'MISS', 'GOAL', 'BLOCK']
correction = ['Tip-In', 'Wrap-around', 'Deflected']
even_strength = ['5v5','4v4','3v3']
uneven_strength = ['5v4','4v5','5v3','3v5','4v3','3v4','5vE','Ev5','4vE','Ev4','3vE','Ev3']

In [43]:
df['event_distance'] = ((89 - abs(df['coords_x']))**2 + df['coords_y']**2)**(1/2)
df['event_angle'] = abs(np.arctan(df['coords_y'] / (89 - abs(df['coords_x']))) * (180 / np.pi))
df['pre_event_zone'] = np.select(conditions, choices, default=np.nan)
df['pre_event_zone_2'] = np.where((df.pre_event_zone == 'Def') & 
                                  (df.event_type == 'BLOCK'), 
                                  'Off',df.pre_event_zone)
df['pre_event_zone_3'] = np.where((df.event_type.isin(fenwick)) & 
                                  (df.event_zone == 'Def') & 
                                  (df.pbp_distance <= 64),
                                  'Off',df.pre_event_zone)
df['event_distance_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x < 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     (((abs(df.coords_x)+89)**2) + df.coords_y**2)**(1/2)
                                     ,df.event_distance)
df['event_distance_long_2'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x > 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     (((df.coords_x+89)**2) + df.coords_y**2)**(1/2)
                                     ,df.event_distance)
df['event_angle_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x < 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     abs(np.arctan(df['coords_y']/(abs(df['coords_x'])+89))*(180 / np.pi))
                                     ,df.event_angle)
df['event_angle_long_2'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x > 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     abs(np.arctan(df['coords_y']/(df['coords_x']+89))*(180 / np.pi))
                                     ,df.event_angle)
df['event_zone_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.event_zone == 'Def') &
                                     (df.pbp_distance <= 64),
                                     'Off',
                                     df.event_zone)
df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     'Ev1',
                                     df.game_strength_state)
df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     '1vE',
                                     df.game_strength_state)
df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     1,
                                     df.home_skaters)
df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     0,
                                     df.home_skaters)
df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     0,
                                     df.away_skaters)
df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     1,
                                     df.away_skaters)

In [44]:
event_interest = ["FAC", "GOAL", "BLOCK", "SHOT", "MISS", "HIT", "TAKE", "GIVE"]
filter_1 = df.event_type.isin(event_interest)
filter_2 = df.game_period < 5
filter_3 = ~df.event_description.str.contains('penalty shot', na=False)
#filter_4 = df.coords_x.isna()
#filter_5 = df.coords_y.isna()
filter_4 = ~((df.coords_x == 0) &
             (df.coords_y == 0) &
             (df.event_type.isin(corsi)) &
             ((df.pbp_distance != 90) & (df.event_type.isin(fenwick))))

In [45]:
#df_2 = df[filter_1 & filter_2 & filter_3 & filter_4 & filter_5 & filter_6].copy()
df = df[filter_1 & filter_2 & filter_3 & filter_4].copy()

In [46]:
df.dropna(subset=['coords_x','coords_y'],inplace=True)

In [47]:
#df = df.groupby(['season', 'game_id', 'game_period']).apply(lambda x: x.sort_values(['event_index']))
df.sort_values(['season', 'game_id', 'game_period','event_index'],inplace=True)

In [48]:
df['seconds_since_last'] = df.game_seconds - df.game_seconds.shift(1)
df['event_type_last'] = df.event_type.shift(1)
df['event_team_last'] = df.event_team.shift(1)
df['event_strength_last'] = df.game_strength_state.shift(1)
df['coords_x_last'] = df.coords_x.shift(1)
df['coords_y_last'] = df.coords_y.shift(1)

In [49]:
filter_5 = df.event_type.isin(fenwick)
filter_6 = df.game_strength_state.isin(even_strength)

In [50]:
df = df[filter_5 & filter_6].copy()
df.dropna(subset=['coords_x','coords_y'],inplace=True)

In [51]:
df['same_team_last']     = np.where(df.event_team == df.event_team_last,1,0)
df['is_home']            = np.where(df.event_team == df.home_team,1,0)
df['score_state']        = np.where(df.is_home == 1, 
                                    df.home_score - df.away_score, 
                                    df.away_score - df.home_score)
df['event_detail']       = np.where(df.event_detail.isna(), 'Wrist', df.event_detail)
df['distance_from_last'] = (((df.coords_x - df.coords_x_last)**2) + 
                            ((df.coords_y - df.coords_y_last)**2))**(1/2)

In [52]:
df.rename(columns={"event_distance": "shot_distance", "event_angle": "shot_angle"}, inplace=True)

In [53]:
keeper_lst = ['game_id', 'event_index', 'season', 'game_date', 'game_period', 'game_seconds', 
              'game_strength_state', 'game_score_state', 'score_state', 'is_home', 
              'event_player_1', 'home_goalie', 'away_goalie', 
              'home_score', 'away_score', 'home_team', 'away_team', 'home_skaters', 'away_skaters', 
              'event_description', 'event_team', 'event_type', 'event_detail', 
              'coords_x', 'coords_y', 'pbp_distance', 'shot_distance', 'shot_angle',
              'event_team_last', 'same_team_last', 'event_strength_last', 'event_type_last', 
              'seconds_since_last', 'distance_from_last', 'coords_x_last', 'coords_y_last']

df = df[keeper_lst].copy()

In [54]:
df['is_goal'] = np.where(df.event_type == 'GOAL',1,0)
df['state_5v5'] = np.where(df.game_strength_state == '5v5',1,0)
df['state_5v4'] = np.where(df.game_strength_state == '5v4',1,0)
df['state_5v3'] = np.where(df.game_strength_state == '5v3',1,0)
df['score_down_4'] = np.where(df.score_state <= 4,1,0)
df['score_down_3'] = np.where(df.score_state == 3,1,0)
df['score_down_2'] = np.where(df.score_state == 2,1,0)
df['score_down_1'] = np.where(df.score_state == 1,1,0)
df['score_even'] = np.where(df.score_state == 0,1,0)
df['score_up_1'] = np.where(df.score_state == 1,1,0)
df['score_up_2'] = np.where(df.score_state == 2,1,0)
df['score_up_3'] = np.where(df.score_state == 3,1,0)
df['score_up_4'] = np.where(df.score_state >= 4,1,0)
df['wrist_shot'] = np.where(df.event_detail == 'Wrist',1,0)
df['deflected_shot'] = np.where(df.event_detail == 'Deflected',1,0)
df['tip_shot'] = np.where(df.event_detail == 'Tip-In',1,0)
df['slap_shot'] = np.where(df.event_detail == 'Slap',1,0)
df['backhand_shot'] = np.where(df.event_detail == 'Backhand',1,0)
df['snap_shot'] = np.where(df.event_detail == 'Snap',1,0)
df['wrap_shot'] = np.where(df.event_detail == 'Wrap-around',1,0)
df['prior_shot_same'] = np.where((df.event_type_last == 'SHOT') & (df.same_team_last == 1),1,0)
df['prior_miss_same'] = np.where((df.event_type_last == 'MISS') & (df.same_team_last == 1),1,0)
df['prior_block_same'] = np.where((df.event_type_last == 'BLOCK') & (df.same_team_last == 1),1,0)
df['prior_shot_opp'] = np.where((df.event_type_last == 'SHOT') & (df.same_team_last == 0),1,0)
df['prior_miss_opp'] = np.where((df.event_type_last == 'MISS') & (df.same_team_last == 0),1,0)
df['prior_block_opp'] = np.where((df.event_type_last == 'BLOCK') & (df.same_team_last == 0),1,0)
df['prior_give_opp'] = np.where((df.event_type_last == 'GIVE') & (df.same_team_last == 0),1,0)
df['prior_give_same'] = np.where((df.event_type_last == 'GIVE') & (df.same_team_last == 1),1,0)
df['prior_take_opp'] = np.where((df.event_type_last == 'TAKE') & (df.same_team_last == 0),1,0)
df['prior_take_same'] = np.where((df.event_type_last == 'TAKE') & (df.same_team_last == 1),1,0)
df['prior_hit_opp'] = np.where((df.event_type_last == 'HIT') & (df.same_team_last == 0),1,0)
df['prior_hit_same'] = np.where((df.event_type_last == 'HIT') & (df.same_team_last == 1),1,0)
df['prior_face'] = np.where(df.event_type_last == 'FAC',1,0)

In [55]:
final_lst = ['game_period','game_seconds','is_home','coords_x', 'coords_y',
             'coords_x_last', 'coords_y_last','shot_distance', 'shot_angle',
             'state_5v5', 'state_5v4', 'state_5v3', 'score_down_4', 'score_down_3',
             'score_down_2', 'score_down_1', 'score_even', 'score_up_1',
             'score_up_2', 'score_up_3', 'score_up_4', 'wrist_shot',
             'deflected_shot', 'tip_shot', 'slap_shot', 'backhand_shot', 'snap_shot',
             'wrap_shot', 'prior_shot_same', 'prior_miss_same', 'prior_block_same',
             'prior_shot_opp', 'prior_miss_opp', 'prior_block_opp', 'prior_give_opp',
             'prior_give_same', 'prior_take_opp', 'prior_take_same', 'prior_hit_opp',
             'prior_hit_same', 'prior_face', 'is_goal']
df = df[final_lst].copy()

In [56]:
df.shape

(90314, 42)

In [57]:
#df = pd.read_csv('data/pbp/raw/pbp_query_20192020.csv')
df.to_csv('data/pbp/ready_to_model/17_18_pbp_cleaned.csv')